In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

### def developer(desarrolladora : str)

In [5]:
def developer (desarrollador):
    
    # Importa el archivo 
    df_dev = pd.read_parquet('def_developer.parquet')

    if desarrollador in df_dev['developer'].values:

        #Se crea un dataframe  con los datos del desarrollador
        dev = df_dev[df_dev['developer']==desarrollador]

        # Se crean las columnas 'free' y 'tot' para hacer el conteo de items
        dev['free']=dev['price'].apply(lambda x: 1 if x == 0 else 0) # 1 para cada item Free
        dev['tot']=1 # 1 para todos los items

        # Se agrupan los registros por desarrollador y año. Se suma la cantidad de items Free y se cuentan los Total
        dev = dev.groupby(['developer','year']).agg({'free': 'sum','tot':'count'}).reset_index()
        
        # Se calcula el porcentaje de contenido Free
        dev['percentage']=((dev['free']/dev['tot'])*100).round(2)

        # Se borran las columnas auxiliares y se cambian los nombres de las que contienen el resultado
        dev.drop(columns=['developer','free'],inplace=True)
        dev.rename(columns={'release_year':'año','tot':'Cantidad de items','percentage':'Contenido Free (%)'},inplace=True)
        
        return dev # Se devuelve el DataFrame
    
    else:

        return f'Desarrollador {desarrollador} no encontrado'

In [6]:
developer('DL Softworks')

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1900\829570361.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev['free']=dev['price'].apply(lambda x: 1 if x == 0 else 0) # 1 para cada item Free
C:\Users\Usuario\AppData\Local\Temp\ipykernel_1900\829570361.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev['tot']=1 # 1 para todos los items


,year,Cantidad de items,Contenido Free (%)
0,2014,2,0.00
1,2015,2,0.00
2,2016,41,9.76
3,2017,33,0.00


In [7]:
developer('Ashes')

'Desarrollador Ashes no encontrado'

### def UserForGenre( genero : str )

In [13]:
def UserForGenre (genero):

    # Importa el archivo
    df_user_gen = pd.read_parquet('def_userforgenre.parquet')
        # Columnas:
            # user_id
            # item_id
            # playtime_forever
            # genres
            # year

    # Se crea un DataFrame con los registros que contienen en su lista de generos el genero ingresado por parametro
    df_gen = df_user_gen[df_user_gen['genres'].apply(lambda x: genero in x)]

    # Se agrupan los registros por usuario y se suman los tiempos de juego
    tot = df_gen.groupby(['user_id']).agg({'playtime_forever': 'sum'}).reset_index()

    if tot.size > 0:
        # Se ordenan los tiempos de juego y se toma el usuario correspondiente al mayor valor
        u_most = tot.sort_values(by='playtime_forever',ascending=False).iloc[0,0]
        
        # Se filtra el DataFrame que contenia solo los registros del genero indicado para el usuario encontrado
        df_user = df_gen[df_gen['user_id']==u_most]

        # Se agrupan los resultados por año
        df_user = df_user.groupby(['year']).agg({'playtime_forever': 'sum'}).reset_index()
        
        # Se renombran las columnas y se crea el diccionario para devolver
        df_user.rename(columns={'year':'Año','playtime_forever':'Horas'},inplace=True)
        res_dict = df_user.to_dict(orient='records')

        return {f"Usuario con más horas jugadas para Genero '{genero}'" : u_most, 
                "Horas jugadas":res_dict}
    
    else:

        return f"No hay registros de horas de juego para Genero '{genero}'"

In [14]:
UserForGenre('Photo Editing')

{"Usuario con más horas jugadas para Genero 'Photo Editing'": 'MarucTp',
 'Horas jugadas': [{'Año': 2006, 'Horas': 2.6666666666666665}]}

In [15]:
UserForGenre('Adventure')

{"Usuario con más horas jugadas para Genero 'Adventure'": 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': 2003, 'Horas': 0.0},
  {'Año': 2008, 'Horas': 20.683333333333334},
  {'Año': 2009, 'Horas': 268.2166666666667},
  {'Año': 2010, 'Horas': 114.83333333333333},
  {'Año': 2011, 'Horas': 816.6999999999999},
  {'Año': 2012, 'Horas': 1295.9666666666667},
  {'Año': 2013, 'Horas': 2436.4},
  {'Año': 2014, 'Horas': 1647.1166666666668},
  {'Año': 2015, 'Horas': 8600.666666666666},
  {'Año': 2016, 'Horas': 9836.016666666666},
  {'Año': 2017, 'Horas': 522.1166666666667}]}